In [5]:
import bempp.api 
import numpy as np
import mtf

from mtf.config import config
from bempp.api.operators.boundary.sparse import identity
from mtf.operators.maxwell import assemble_mtf
from mtf.utils.iterative import gmres
from mtf.functions import define_bempp_functions

from matplotlib import pyplot as plt
bempp.api.enable_console_logging()
import argparse

parser = argparse.ArgumentParser(description="Set parameters")

parser.add_argument("--M", default=3, type=int)
parser.add_argument("--solver", default='gmres', type=str)
parser.add_argument("--precision", default=10, type=int)
parser.add_argument("--case", default="B", type=str)
args = parser.parse_args([])

M = args.M
solver = args.solver
case = args.case
precision = args.precision

M = 3
case = 'B'
shape = 'sphere'
precision = 20

print('Solver is: ', solver)
config = mtf.config.set_case(case, False)
k0 = config['k_ext']
k1 = config['k_int']

if M == 2:
    #multiple shape, M=2
    segments = [[10, 20], [10, 20]]
    swapped_normals = [[10, 20], []]
elif M == 3:
    #multiple shape, M=3
    segments = [[10, 20], [10, 12], [12, 20]]
    swapped_normals = [[10, 20], [], [12]]

## Same for the multishape
if shape == 'cube':
    segments = [[1, 2, 3, 4, 5, 7, 8, 9, 10, 11],
                [1, 2, 3, 4, 5, 6],
                [6, 7, 8, 9, 10, 11]]
    
    swapped_normals = [[1, 2, 3, 4, 5, 7, 8, 9, 10, 11],[],[6]]

# Outside: k_ext
# Inside: k_int
# Cjecl general 

k_list = [k0]
eta_rel_list = [1]
mu_list = [config['mu_ext']]

for index in range(M-1):
  k_list.append(k1)
  mu_list.append(config['mu_int'])
  eta_rel_list.append(config['eta_rel'])

print("k_ext: {:.2f}".format(config['k_ext']))
print("k_int: {:.2f}".format(config['k_int']))

print("----")
print("lambda: {:.2f}".format(config['lambda']))
print("f: {:.2E}".format(config['frequency']))

h = 2 * np.pi/(precision*k0)
if shape == 'sphere':
    grid = bempp.api.shapes.multitrace_sphere(h=h)
elif shape == 'cube':
    grid = bempp.api.shapes.multitrace_cube(h=h)
else:
    print('choose a valid shape')

params = {}
params["M"] = M
params["k_list"] = k_list
params["mu_list"] = mu_list
params["eta_rel_list"] = eta_rel_list
params["segments"] = segments
params["swapped_normals"] = swapped_normals


if solver == 'gmres':
    lhs_prec, lhs_op, rhs = assemble_mtf(grid, params, config, solver=solver)
    P_wf = lhs_prec.weak_form()
else:
    lhs_op, rhs = assemble_mtf(grid, params, config, solver=solver)

# Solver

op_wf = lhs_op.weak_form()
op_sf = lhs_op.strong_form()

b_wf = bempp.api.assembly.blocked_operator.projections_from_grid_functions_list(
    rhs, lhs_op.dual_to_range_spaces
)


N = b_wf.shape[0]
print(N)
x_gmres, conv_gmres, res_gmres = gmres(P_wf * op_wf, P_wf * b_wf, return_residuals=True, restart = 2000, maxiter=1500)
x_unprec, conv_unprec, res_unprec = gmres(op_wf, b_wf, return_residuals=True, restart = 2000, maxiter=2000)

sol_gmres = bempp.api.assembly.blocked_operator.grid_function_list_from_coefficients(
    x_gmres.ravel(), lhs_op.domain_spaces
)

results_conv = np.array([res_gmres])
results_unprec = np.array([res_unprec])

name_conv = "results/" + str(M) + case + str(precision) + "mtf" + solver + 'gmres' + str(shape)
name_unprec = "results/" + str(M) + case + str(precision) + "mtf" + solver + 'gmres_unprec' + str(shape)

np.save(name_conv, results_conv)
np.save(name_unprec, results_unprec)

Solver is:  gmres
[3.0, 4.347413023856832, 4.347413023856832]
[1.2566e-06, 1.2566e-06, 1.2566e-06]
[1, 0.6900655593423543, 0.6900655593423543]


In [ ]:
res0 = np.load("results/" + str(M) + case + str(10) + "mtf" + solver + 'gmres' + str(shape) +'.npy').ravel()
res1 = np.load("results/" + str(M) + case + str(20) + "mtf" + solver + 'gmres' + str(shape) + '.npy').ravel()

res0_unprec = np.load("results/" + str(M) + case + str(10) + "mtf" + solver + 'gmres_unprec' + str(shape) + '.npy').ravel()
res1_unprec = np.load("results/" + str(M) + case + str(20) + "mtf" + solver + 'gmres_unprec' + str(shape) + '.npy').ravel()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

ax.plot(np.log10(res0), label = r'$N_0$', color = 'b')
ax.plot(np.log10(res1), label = r'$N_1$', color = 'b', linestyle='--')

ax.plot(np.log10(res0_unprec), label = r'$N_0$', color = 'k')
ax.plot(np.log10(res1_unprec), label = r'$N_1$', color = 'k', linestyle = '--')


ax.legend()
plt.grid(linestyle='--')
ax.set_xlabel('Iteration count')
ax.set_ylabel('Relative residual error')
plt.savefig('s2_convergence' + str(shape) + '.pdf')
plt.show()